In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
import mdtraj
import urllib.request
import yaml


sys.path.insert(1, '../../Databank/Scripts/BuildDatabank/')
from databankLibrary import download_link, lipids_dict, databank

path = '../../Databank/Data/Simulations/'
db_data = databank(path)
systems = db_data.get_systems()

def CalcAreaPerMolecule(system):
    APLpath = system['path'] + 'apl.json'
    try:
        f = open(APLpath)
        APLdata = json.load(f)
        sumAPL = 0
        sumIND = 0
        for i,j in APLdata.items():
            sumAPL += j
            sumIND += 1
        APL = sumAPL/sumIND
        return(APL)
    except:
        print('apl.json not found from' + system['path'])
        
def GetNlipids(system):
    Nlipid = 0
    for molecule in system['COMPOSITION']:
        if molecule in lipids_dict:
            Nlipid += np.sum(system['COMPOSITION'][molecule]['COUNT'])
    return Nlipid


In [2]:
InterestingComposition = ['POPC', 'CHOL','POPE','POPS','SAPI']
#InterestingComposition = ['POPS', 'POPE','POPC']
aplArray = []
interestArray = []

for system in systems:
    skip = True
    
    for interest in InterestingComposition:
        if interest in system['COMPOSITION']:
            skip = False
    
    if skip:
        continue

        
    interestArrayTMP = []
    for interest in InterestingComposition:
        try:
            interestArrayTMP.append(sum(system['COMPOSITION'][interest]['COUNT']) / GetNlipids(system))
        except:
            interestArrayTMP.append(0)
    
    aplArray.append(CalcAreaPerMolecule(system))
    
    interestArray.append(interestArrayTMP)
    
    #print(system['COMPOSITION'])
    #print(CalcAreaPerMolecule(system))
    #print()

In [3]:
from sklearn.linear_model import LinearRegression

X = np.array(interestArray)
y = np.array(aplArray)

# Create an instance of the LinearRegression class
reg = LinearRegression()
 
# Fit the model to the data
reg.fit(X, y)
 
# Print the coefficients of the model
print(reg.coef_)

[ -1.13077855 -45.22393764  -5.29227884  -7.25145393 -18.6417378 ]


In [6]:
reg.predict([[0.35, 0.25,0.25,0.1,0.05]])
#reg.predict([[0.8, 0.15,0.05]])

array([49.29803795])